# Differential gene expression for samples from BeatAML study*

The analysis is based on two datasets from a study by Tyner et al. (2018). The first one can be obtained from http://www.vizome.org/aml (_Sample data_ -> _All sample data_). It contains rich data on tumor samples from patients suffering from AML (Acute Myeloid Leukemia). See the original paper as well as _Data dictionary_ on the provided website. The dataset is already saved in the input directory as _`BeatAML_RNASeq_rawcounts_2018_10_24.csv.gz`_.

The second dataset can be obtained from http://vizome.org/additional_figures_BeatAML.html as _Full Cohort RNASeq Counts_. It contains raw read counts for samples subjected to RNA-Seq analysis. The dataset is already saved in the input directory as _`BeatAML_all_sample_data.csv.gz`_.

Utilise those two datasets by taking subsequent steps in order to perform differential gene expression (DGE) between two groups: samples from patients with or without a mutation in NPM1 gene. The logic behind the analysis is concisely explained step by step.

---

<font size="1">*Tyner JW, Tognon CE, Bottomly D, Wilmot B, Kurtz SE, Savage SL, Long N, Schultz AR, Traer E, Abel M, Agarwal A. Functional genomic landscape of acute myeloid leukaemia. Nature. 2018 Oct 25;562(7728):526-31.</font>

---

- Import `DESeq2` library.

In [1]:
library("DESeq2")

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb

Loading required package: SummarizedExperiment

Loading required package: M

- Load BeatAML sample data on all samples to `smplData` data frame. Select only those rows for which `RNASeq` column values are `TRUE`.

In [2]:
smplData <- read.csv("input/BeatAML_all_sample_data.csv.gz", header=TRUE, check.names=FALSE, skip=3)
smplData <- smplData[smplData[, "RNAseq"],]

- Get sample IDs (`LLS_SampleID` column) and check for which the column `SpecificDxAtInclusion` (diagnosis at inclusion) indicates samples from patients with AML diagnosis with mutated NPM1 gene.

In [3]:
smpls   <- smplData[, "LLS_SampleID"]
mutated <- smplData[, "SpecificDxAtInclusion"] == "AML with mutated NPM1"

- Create a data frame `metaData` describing samples for DGE. Row names denote sample IDs, the `groups` column contains the following labels: `NPM1norm` for samples without the mutation diagnosed, `NPM1mut` for samples with the mutation diagnosed. The type of this column is set to `factor` and `NPM1norm` as the reference group.

In [4]:
groups   <- rep("NPM1norm", length(smpls))
groups[mutated] <- "NPM1mut"
groups   <- factor(groups)
groups   <- relevel(groups, ref="NPM1norm")
metaData <- data.frame(row.names=smpls, groups)

- Look up the dimensions and first 5 rows of the resulting metaData data frame.

In [5]:
dim(metaData)
head(metaData)

[1] 507   1

,groups
,<fct>
15-00246,NPM1mut
17-00040,NPM1norm
14-00514,NPM1norm
15-00981,NPM1norm
13-00150,NPM1norm
14-00376,NPM1norm


- Look the number of samples in each group.

In [6]:
table(metaData[, "groups"])


NPM1norm  NPM1mut 
     400      107 

- Load BeatAML raw gene counts to the countData data frame, look up the dimensions and the first 5 rows.

In [7]:
countData <- read.csv("input/BeatAML_RNASeq_rawcounts_2018_10_24.csv.gz", header=TRUE, check.names=FALSE, row.names=1)
dim(countData)
head(countData)

[1] 63677   502

,Symbol,Chr,Exon_Start,Exon_End,Strand,Length,GeneStart,GeneEnd,12-00023,12-00051,⋯,17-00043,17-00044,17-00045,17-00046,17-00047,17-00049,17-00051,17-00052,17-00053,17-00055
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
ENSG00000000003,TSPAN6,X,99883667;99885756;99887482;99888402;99888928;99890175;99890555;99891188;99894942,99884983;99885863;99887565;99888536;99889026;99890249;99890743;99892101;99894988,-,2968,99883667,99894988,8,52,⋯,41,16,18,28,35,34,35,27,34,20
ENSG00000000005,TNMD,X,99839799;99840228;99848621;99849258;99852501;99854013;99854505,99840063;99840359;99849032;99849359;99852654;99854179;99854882,+,1610,99839799,99854882,0,0,⋯,0,1,0,1,0,0,0,0,0,0
ENSG00000000419,DPM1,20,49551404;49552685;49557402;49557642;49558568;49562274;49562384;49565166;49571723;49574900,49551773;49552799;49557492;49557746;49558663;49562299;49562460;49565199;49571822;49575092,-,1207,49551404,49575092,1080,920,⋯,1087,1208,1058,1297,1069,1163,1041,1366,1389,1198
ENSG00000000457,SCYL3,1,169818772;169823411;169824937;169828182;169831754;169833510;169836037;169838069;169839396;169842837;169845119;169847775;169857817;169862929;169863148,169822913;169824105;169825098;169828353;169831938;169833649;169836114;169838269;169839498;169842893;169845232;169847960;169858031;169863093;169863408,-,6876,169818772,169863408,701,587,⋯,881,990,927,1008,706,996,791,865,651,751
ENSG00000000460,C1orf112,1,169631245;169652610;169652897;169752952;169754018;169763871;169764181;169764550;169767998;169770024;169771762;169772310;169773216;169775145;169776932;169790820;169792549;169795234;169796192;169796852;169798404;169799400;169799837;169801558;169806068;169811558;169812815;169816831;169818641;169819406;169819595;169820958;169821931,169631761;169652766;169653073;169753069;169754054;169764046;169764354;169765124;169768099;169770112;169771866;169772450;169773521;169775229;169777070;169790900;169792613;169795258;169796340;169796981;169798955;169799482;169799888;169801647;169806253;169811680;169812910;169816944;169818745;169819486;169819707;169821077;169823221,+,6354,169631245,169823221,249,82,⋯,560,548,511,569,459,421,625,444,667,416
ENSG00000000938,FGR,1,27938575;27939730;27940941;27941361;27941945;27942200;27943368;27943704;27948070;27949553;27949895;27951600;27952557;27952971;27961576,27939633;27939861;27941094;27941437;27942124;27942355;27943517;27943807;27948168;27949655;27950573;27951662;27952751;27953080;27961788,-,3474,27938575,27961788,29052,541,⋯,21732,26652,24034,18079,25757,21527,20409,27210,1604,22325


- Select columns the names of which match sample ID pattern, look up the dimensions and the first 5 rows.

In [8]:
countData <- countData[, grepl("^[0-9]+\\-[0-9]+$", colnames(countData))]
dim(countData)
head(countData)

[1] 63677   494

,12-00023,12-00051,12-00066,12-00150,12-00211,12-00258,12-00294,12-00301,12-00372,12-00423,⋯,17-00043,17-00044,17-00045,17-00046,17-00047,17-00049,17-00051,17-00052,17-00053,17-00055
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
ENSG00000000003,8,52,2,60,0,0,0,1,32,18,⋯,41,16,18,28,35,34,35,27,34,20
ENSG00000000005,0,0,0,0,0,0,0,0,0,0,⋯,0,1,0,1,0,0,0,0,0,0
ENSG00000000419,1080,920,664,1128,747,921,955,324,797,1019,⋯,1087,1208,1058,1297,1069,1163,1041,1366,1389,1198
ENSG00000000457,701,587,300,624,1120,491,1198,279,1172,760,⋯,881,990,927,1008,706,996,791,865,651,751
ENSG00000000460,249,82,317,395,373,257,577,133,386,487,⋯,560,548,511,569,459,421,625,444,667,416
ENSG00000000938,29052,541,25829,1720,3326,31518,8188,13510,5928,9186,⋯,21732,26652,24034,18079,25757,21527,20409,27210,1604,22325


- Find the intersection of sample IDs from sample data (`metaData`) and count data (`countData`), look up the number of obtained sample IDs and the first 5 of them.

In [9]:
common <- intersect(rownames(metaData), colnames(countData))
length(common)
common[1:5]

[1] 493

[1] "15-00246" "17-00040" "14-00514" "15-00981" "13-00150"

- From the `metaData` and `countData` data frames select respectively rows and columns that correspond to sample IDs common for both data frames, look up dimensions of the resulting data frames.

In [10]:
metaData  <- metaData[common, , drop=FALSE]
countData <- countData[, common, drop=FALSE]
dim(metaData)
dim(countData)

[1] 493   1

[1] 63677   493

- Using `metaData` and `countData` data frames, create a DESEq2 dataset (`dss`). Get rid of rows with less than total 10 counts and perform DGE.

In [11]:
dds <- DESeqDataSetFromMatrix(countData=countData, colData=metaData, design=~groups)
dds <- dds[ rowSums( counts(dds) ) >= 10, ]
dds <- DESeq(dds)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 4541 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



- Obtain results with the FDR (false discovery rate, `alpha`) threshold set to `0.01` and perform `log2FoldChange` shirnkage using _apegml_ method.

In [12]:
res  <- results(dds, alpha=0.01)
coef <- resultsNames(dds)[2]
res  <- lfcShrink(dds, res=res, coef=coef, type="apeglm")

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895



- Look up the result summary.

In [13]:
summary(res)


out of 42142 with nonzero total read count
adjusted p-value < 0.01
LFC > 0 (up)       : 2747, 6.5%
LFC < 0 (down)     : 5022, 12%
outliers [1]       : 0, 0%
low counts [2]     : 9811, 23%
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



- Check which rows (gene IDs) were skipped prior performing the DGE analysis, add them to final results and fill with NA values.

In [14]:
skipped <- setdiff(rownames(countData), rownames(res))
res[skipped,] <- NA

- Create a data frame with gene IDs as the first column and DGE results as remianing. Save the complete and final results to a TSV file.

In [15]:
res_df <- cbind(rownames(res), data.frame(res, row.names=NULL))
colnames(res_df)[1] <- "gene_id"

dir.create("output", recursive=TRUE)
write.table(res_df, file="output/DGE_results.tsv", sep="\t", row.names=FALSE, quote=FALSE)

Warning message in dir.create("output", recursive = TRUE):
“'output' already exists”
